# part 2- Data Cleaning

here we clean the wildfire data

In [1]:
import bs4
import datetime as dt
import time
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# import requests
import pandas as pd
from bs4 import BeautifulSoup
from collections import defaultdict

# Upload CSV

In [2]:
fullCSV="wildfiresFULLDATA.csv"
BeforeCleaning= "wildfiresBeforeCleaning.csv"
after_cleaningCSV="wildfiresAfterCleaning.csv"


In [3]:
#upload csv to dateframe
def load_csv(csv):
    return pd.read_csv(csv)

In [4]:
#remove all duplicate rows
def remove_duplicatives(df, col_name=None):
    if col_name != None :
        return df.drop_duplicates(subset = [col_name],keep = 'first' ).copy()
    else:
        return df.drop_duplicates(keep ='first').copy()

In [5]:
#remove all NAN
def removeNan(df):
    df1 = df.dropna(axis=0)
    return df1


In [6]:
#creating BeforeCleaning csv with the wanted cols and without missing data
def removecols_androws(df):
    df_remove_cols = df[['UniqueFireIdentifier','IncidentName','InitialLatitude','InitialLongitude','FireDiscoveryDateTime','FireOutDateTime','FireCause','DiscoveryAcres']]
    df_remove_cols = df_remove_cols.dropna(subset=['UniqueFireIdentifier','IncidentName','InitialLatitude','InitialLongitude','FireDiscoveryDateTime','FireOutDateTime','FireCause','DiscoveryAcres'])
    df_remove_cols.to_csv(BeforeCleaning)

In [7]:
#to make it easy for us with col:firecause we make it to 1,2,3,4
def convert_Firecause(df):
    df1 = df.copy()
    df1['FireCause'] = df1['FireCause'].replace({'Human': 1, 'Natural': 2,'Unknown': 3,'Undetermined': 4})
     # שהשארנו הכל אחוזי הנכונות של המודל היה 68 אחוז.
     # שהשארנו רק את השריפות שנגרמו על ידי בני אדם ומזג האוויר הנכונות של המודל היה 98 אחוז.
     # שהוצאנו רק את השריפות שלא ידוע מה גורם השריפה קיבלנו אחוזי נכונות של 73 אחוז וזה המודל הנוכחי.
    df1 = df1[df1['FireCause']!=3]
    return df1


In [8]:
#to merge from weather CSV we need this format
def convert_Date(df):
    df1 = df.copy()
    df1['FireDiscoveryDateTime'] = pd.to_datetime(df1['FireDiscoveryDateTime']).dt.strftime('%Y-%m-%dT%H:%M:%S')
    df1.drop([71505], axis=0, inplace=True)
    df1['FireOutDateTime'] = pd.to_datetime(df1['FireOutDateTime']).dt.strftime('%Y-%m-%dT%H:%M:%S')
    return df1


# Dealing with outliers


we checked the range latitude and longtitude of usa
and after this phase we stay only with the good datum-points


In [9]:
#latitude outlier
def outlierLat(df):
    df_copy = df.copy()
    Q1 = np.percentile(df_copy["InitialLatitude"], 25)
    Q3 = np.percentile(df_copy["InitialLatitude"], 75)
    IQR = Q3 - Q1
    df.InitialLatitude[(df["InitialLatitude"] < Q1 - 1.5 * IQR) | (df["InitialLatitude"] > Q3 + 1.5 * IQR)] = np.nan
    return df

In [10]:
#longtitude outlier
def outlierLong(df):
    df_copy = df.copy()
    z_score = (df_copy["InitialLongitude"] - df_copy["InitialLongitude"].mean()) / df_copy["InitialLongitude"].std()
    outliers = abs(z_score) > 3
    sum(outliers)
    df.InitialLongitude[outliers] = np.nan
    # delete rows above 1000 in DiscoveryAcres
    df_copy = df_copy[df_copy['InitialLongitude'] < -30]
    df = df_copy[df_copy['InitialLongitude'] > -160]
    return df

In [11]:
#create binary target col for machine learning
def CausedByWeatherCol(df):
    CausedByWeather = []
    for row in df.index:
        if df['FireCause'][row] == 2:
            CausedByWeather.append(1)
        else:
            CausedByWeather.append(0)
    return CausedByWeather

In [12]:
#summon all cleaning functions for new csv
def csv_After_Cleaning(df):
    df1 = df.copy()
    df1 = outlierLat(df1)
    df1 = outlierLong(df1)
    df1 = removeNan(df1)
    df1 = convert_Date(df1)
    df1 = convert_Firecause(df1)
    df1['CausedByWeather'] = CausedByWeatherCol(df1)
    df1.to_csv(after_cleaningCSV)

In [13]:
#full csv
df = load_csv(fullCSV)
df

C:\Users\HEN\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (13,14,16,32,61,64,68,69,77,78,94,95) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,X,Y,OBJECTID,ABCDMisc,ADSPermissionState,CalculatedAcres,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,...,IsDispatchComplete,OrganizationalAssessment,StrategicDecisionPublishDate,CreatedOnDateTime_dt,ModifiedOnDateTime_dt,Source,GlobalID,IsCpxChild,CpxName,CpxID
0,-104.457511,45.785037,1,NaN,CERTIFIED,50.64,2020/08/06 23:13:07+00,2020/08/06 23:13:24+00,50.60,20.00,...,0,NaN,NaN,2020/08/06 19:50:29+00,2020/08/12 20:46:01+00,IRWIN,{E5436898-ED0D-4CB1-90C0-D61915FE1F29},NaN,NaN,NaN
1,-118.180712,33.808985,2,NaN,DEFAULT,NaN,NaN,NaN,NaN,0.10,...,0,NaN,NaN,2020/02/28 20:52:36+00,2020/02/28 20:52:36+00,IRWIN,{0E79B7FD-2882-43CF-8CFA-911BD1C8F77A},NaN,NaN,NaN
2,-114.835414,48.073946,3,NaN,DEFAULT,NaN,2017/10/18 00:30:00+00,2017/10/18 00:35:00+00,50.00,50.00,...,0,NaN,NaN,2017/10/18 13:46:40+00,2017/11/09 22:08:19+00,IRWIN,{FAC59A92-E6AD-443B-8625-4AAABCF7F533},NaN,NaN,NaN
3,-117.153901,33.176394,4,NaN,DEFAULT,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,2019/07/01 20:10:12+00,2019/07/01 20:10:12+00,IRWIN,{5DF06F41-9948-49D3-B00A-2D3A1D1049C5},NaN,NaN,NaN
4,-121.104180,38.834727,5,NaN,DEFAULT,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,2016/06/20 22:39:02+00,2016/06/20 22:39:02+00,IRWIN,{F378818E-D541-4E0A-9A44-C81886C2B8B4},NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247867,-101.109410,42.762637,293521,NaN,DEFAULT,NaN,NaN,NaN,1.00,1.00,...,0,NaN,NaN,2023/01/06 22:42:13+00,2023/01/06 22:42:15+00,IRWIN,{31D6319F-33DA-4652-AEE1-1EA4C5F75159},0.0,NaN,NaN
247868,-116.181226,33.644994,293522,NaN,DEFAULT,NaN,NaN,NaN,NaN,0.01,...,0,NaN,NaN,2023/01/06 22:43:49+00,2023/01/06 22:43:49+00,IRWIN,{4ABBC3DA-6D6E-4A00-BD6F-DDB797BD4C8A},0.0,NaN,NaN
247869,-116.212353,33.731520,293523,NaN,DEFAULT,NaN,2023/01/06 23:40:00+00,NaN,0.01,0.01,...,0,NaN,NaN,2023/01/06 23:42:02+00,2023/01/06 23:54:03+00,IRWIN,{BF8BC24B-1810-4C32-B54A-FD67A8DDE4D8},0.0,NaN,NaN
247870,-117.311062,33.617268,293524,NaN,DEFAULT,NaN,NaN,NaN,NaN,0.01,...,0,NaN,NaN,2023/01/07 00:05:06+00,2023/01/07 00:05:06+00,IRWIN,{4C8AA093-6036-4108-B33D-D1B544F953BF},0.0,NaN,NaN


In [14]:
removecols_androws(df)


In [15]:
#before finishing cleaning
dfbefore = load_csv(BeforeCleaning)
dfbefore

,Unnamed: 0,UniqueFireIdentifier,IncidentName,InitialLatitude,InitialLongitude,FireDiscoveryDateTime,FireOutDateTime,FireCause,DiscoveryAcres
0,0,2020-MTLG42-000224,Ramme,45.78496,-104.49580,2020/08/06 18:58:00+00,2020/08/12 14:00:00+00,Human,20.00
1,2,2017-MTNWS-000878,South Lost Prairie,48.07167,-114.83030,2017/10/17 20:20:24+00,2017/11/09 21:59:59+00,Human,50.00
2,5,2020-MSMNF-000308,Whatley,31.06819,-89.06972,2020/11/23 19:17:00+00,2020/11/30 14:29:59+00,Human,1.00
3,6,2019-UTUWF-000883,Diamond,40.07631,-111.41820,2019/10/26 21:29:00+00,2019/11/13 00:14:59+00,Unknown,0.10
4,7,2020-MTCES-006641,Alaska Basin,44.65363,-111.56360,2020/08/27 14:06:38+00,2020/08/27 20:52:59+00,Natural,0.10
...,...,...,...,...,...,...,...,...,...
121140,247703,2023-CACDD-000207,DUMP / LV OPEN NEED FIRECODE,33.64565,-114.70080,2023/01/03 08:09:48+00,2023/01/03 21:49:59+00,Undetermined,0.10
121141,247706,2022-MTHLF-000556,Tenmile South Helena Unit 89,46.55692,-112.10170,2022/11/17 18:43:10+00,2022/12/19 15:59:59+00,Undetermined,1.00
121142,247840,2023-ILILS-230011,Havlik PVT RX,37.45070,-88.87618,2023/01/06 16:04:44+00,2023/01/06 18:46:59+00,Human,1.00
121143,247841,2023-TXTXS-000011,MM 104,35.49036,-101.78370,2023/01/06 16:24:00+00,2023/01/06 17:29:00+00,Human,0.25


In [16]:
#after cleaning
csv_After_Cleaning(dfbefore)
dfafter = load_csv(after_cleaningCSV)
dfafter

<ipython-input-9-6b71afbb7f8d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.InitialLatitude[(df["InitialLatitude"] < Q1 - 1.5 * IQR) | (df["InitialLatitude"] > Q3 + 1.5 * IQR)] = np.nan
<ipython-input-10-13f5a2dea539>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.InitialLongitude[outliers] = np.nan


,Unnamed: 0,Unnamed: 0.1,UniqueFireIdentifier,IncidentName,InitialLatitude,InitialLongitude,FireDiscoveryDateTime,FireOutDateTime,FireCause,DiscoveryAcres,CausedByWeather
0,0,0,2020-MTLG42-000224,Ramme,45.78496,-104.49580,2020-08-06T18:58:00,2020-08-12T14:00:00,1,20.00,0
1,1,2,2017-MTNWS-000878,South Lost Prairie,48.07167,-114.83030,2017-10-17T20:20:24,2017-11-09T21:59:59,1,50.00,0
2,2,5,2020-MSMNF-000308,Whatley,31.06819,-89.06972,2020-11-23T19:17:00,2020-11-30T14:29:59,1,1.00,0
3,4,7,2020-MTCES-006641,Alaska Basin,44.65363,-111.56360,2020-08-27T14:06:38,2020-08-27T20:52:59,2,0.10,1
4,5,9,2015-CAYNP-000028,41,37.63880,-119.69320,2015-06-09T03:36:00,2015-06-09T16:57:00,1,0.10,0
...,...,...,...,...,...,...,...,...,...,...,...
109054,121140,247703,2023-CACDD-000207,DUMP / LV OPEN NEED FIRECODE,33.64565,-114.70080,2023-01-03T08:09:48,2023-01-03T21:49:59,4,0.10,0
109055,121141,247706,2022-MTHLF-000556,Tenmile South Helena Unit 89,46.55692,-112.10170,2022-11-17T18:43:10,2022-12-19T15:59:59,4,1.00,0
109056,121142,247840,2023-ILILS-230011,Havlik PVT RX,37.45070,-88.87618,2023-01-06T16:04:44,2023-01-06T18:46:59,1,1.00,0
109057,121143,247841,2023-TXTXS-000011,MM 104,35.49036,-101.78370,2023-01-06T16:24:00,2023-01-06T17:29:00,1,0.25,0
